In [1]:
import numpy as np
import pandas as pd

In [3]:
tableau = pd.read_csv('tableau.csv')
tableau

,order_member_uuid,order_mid,order_create_dt_tw,order_type,order_create_source_code,accounting_price,order_profit
0,6488d197-2c5a-4616-8fed-20bed8da19c1,24KK291105310,2024-07-14,GO,ANDROID,38.73,2.22
1,056eb03b-f794-423e-8efd-da7f157585fa,24KK293105610,2024-07-14,BACK,ANDROID,7.96,0.67
2,2346ce86-dea8-4661-a911-f1d081820d49,24KK290805092,2024-07-14,GO,ANDROID,34.47,0.67
3,e5b6cbf9-d845-46d1-8300-05b3d77ea09d,24KK291105420,2024-07-14,BACK,IOS,28.90,2.34
4,b514a349-995c-4187-a482-62b3fb484afa,24KK291805082,2024-07-14,BACK,IOS,39.93,2.40
...,...,...,...,...,...,...,...
510434,df8b23bb-7ca1-4883-85f4-2dd7cf04ea19,24KK272035013,2024-08-24,BACK,IOS,70.03,13.98
510435,231cfe0a-69fe-4425-bcf3-fbb892d29b07,24KK277635973,2024-08-24,BACK,IOS,4.49,0.90
510436,4200e22a-d5a6-4726-85a7-d404e67fa888,24KK272535033,2024-08-24,BACK,IOS,11.04,2.01
510437,307ea0a0-5857-456c-9cb0-15e6565e1a4b,24KK275535923,2024-08-24,BACK,IOS,10.60,1.92


In [163]:
appsflyer_raw = pd.read_csv('merged_file.csv')

/var/folders/81/8rrb2nqs7t39v9t0vy9jm2s80000gq/T/ipykernel_69294/3071957616.py:1: DtypeWarning: Columns (0,1,9,13,14,15,16,18,20,21,22,23,31,32,33,36,37,39,40,51,53,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  appsflyer_raw = pd.read_csv('merged_file.csv')


In [165]:
appsflyer_raw.shape

(181282, 59)

In [167]:
appsflyer = appsflyer_raw[['Customer User ID','Install Time', 'Media Source', 'Platform']]

In [169]:
appsflyer = appsflyer.dropna(subset='Customer User ID')

In [171]:
appsflyer.columns = ['customer_user_id','install_time', 'media_source', 'platform']

In [173]:
appsflyer = appsflyer.drop_duplicates(subset='customer_user_id')

In [175]:
appsflyer = appsflyer.fillna({'media_source':'organic'})

In [177]:
appsflyer

,customer_user_id,install_time,media_source,platform
0,42627abe-9b98-408a-a146-148c6eb56529,2024-07-31 23:55:37,googleadwords_int,android
1,a833a424-5903-436f-8c4c-5984f9905a36,2024-07-30 17:53:19,googleadwords_int,android
2,96896acd-012d-41eb-ab2a-4db2c9bb4bf3,2024-07-31 23:51:28,googleadwords_int,android
3,2b4e1f45-3fa2-42d6-af72-5563a2bccd41,2024-07-31 23:48:49,Smart_Banner,android
4,af46299b-74d0-401f-98de-06580eb7e1a3,2024-07-31 23:48:49,Smart_Banner,android
...,...,...,...,...
181199,534662c8-d854-486c-a7cd-2129ab4e7c2c,2024-08-01 01:27:20,organic,ios
181205,6331aeed-a7a7-4147-9fc6-d6e0f8d61fd5,2024-08-01 01:16:21,organic,ios
181212,ad1ddc6c-68a4-47fa-9615-70ee696da947,2024-08-01 00:42:14,organic,ios
181216,62207612-d038-425d-bee5-8ccddc952684,2024-08-01 00:57:25,organic,ios


In [179]:
# merge data
app_roi = pd.merge(appsflyer, tableau, how='left',left_on='customer_user_id', right_on='order_member_uuid')

In [181]:
app_roi.install_time = pd.to_datetime(app_roi.install_time)
app_roi.order_create_dt_tw = pd.to_datetime(app_roi.order_create_dt_tw)
app_roi['install_date'] = app_roi.install_time.dt.date

In [183]:
df_filtered = pd.DataFrame()
for uuid, group in app_roi.groupby('customer_user_id'):
    group_filtered = group[(group['order_create_dt_tw'] >= group['install_date']) & 
                           (group['order_create_dt_tw'] <= group['install_date'] + pd.Timedelta(days=30))]
    df_filtered = pd.concat([df_filtered, group_filtered])

In [185]:
df_filtered

,customer_user_id,install_time,media_source,platform,order_member_uuid,order_mid,order_create_dt_tw,order_type,order_create_source_code,accounting_price,order_profit,install_date
16328,00016d3c-dab8-4b2b-8d95-5b2a84a60161,2024-07-03 19:13:05,googleadwords_int,android,00016d3c-dab8-4b2b-8d95-5b2a84a60161,24KK295465466,2024-07-03,BACK,ANDROID,86.20,15.04,2024-07-03
16329,00016d3c-dab8-4b2b-8d95-5b2a84a60161,2024-07-03 19:13:05,googleadwords_int,android,00016d3c-dab8-4b2b-8d95-5b2a84a60161,24KK292365386,2024-07-03,BACK,ANDROID,30.68,6.14,2024-07-03
30371,0001f300-906f-4ba7-bbcb-abe01818b420,2024-07-29 02:18:00,organic,ios,0001f300-906f-4ba7-bbcb-abe01818b420,24KK290285254,2024-08-01,BACK,IOS,23.68,0.07,2024-07-29
30372,0001f300-906f-4ba7-bbcb-abe01818b420,2024-07-29 02:18:00,organic,ios,0001f300-906f-4ba7-bbcb-abe01818b420,24KK290285524,2024-08-01,BACK,IOS,12.42,0.64,2024-07-29
30373,0001f300-906f-4ba7-bbcb-abe01818b420,2024-07-29 02:18:00,organic,ios,0001f300-906f-4ba7-bbcb-abe01818b420,24KK292285014,2024-08-01,BACK,IOS,4.31,0.86,2024-07-29
...,...,...,...,...,...,...,...,...,...,...,...,...
33180,fff9d9f3-c3e1-435e-8146-85df5a6c784d,2024-07-16 20:21:50,organic,ios,fff9d9f3-c3e1-435e-8146-85df5a6c784d,24KK291845707,2024-07-20,BACK,IOS,56.85,0.59,2024-07-16
59115,fffbeb49-f2bc-46c9-94d5-9fbee3a65374,2024-08-05 19:32:19,rd_mkt_web,ios,fffbeb49-f2bc-46c9-94d5-9fbee3a65374,24KK270215777,2024-08-06,BACK,IOS,24.00,-0.82,2024-08-05
59116,fffbeb49-f2bc-46c9-94d5-9fbee3a65374,2024-08-05 19:32:19,rd_mkt_web,ios,fffbeb49-f2bc-46c9-94d5-9fbee3a65374,24KK270215977,2024-08-06,BACK,IOS,21.37,0.89,2024-08-05
28059,fffca042-40e9-45cb-bdd2-0106ef57af6f,2024-07-08 13:42:58,rd_mkt_web,ios,fffca042-40e9-45cb-bdd2-0106ef57af6f,24KK299055724,2024-07-08,BACK,IOS,58.62,2.89,2024-07-08


In [187]:
average_sale_per_user = df_filtered.groupby('customer_user_id').sum(['accounting_price', 'order_profit'])

In [189]:
average_sale_per_user = average_sale_per_user.reset_index()

In [191]:
average_sale_per_user

,customer_user_id,accounting_price,order_profit
0,00016d3c-dab8-4b2b-8d95-5b2a84a60161,116.88,21.18
1,0001f300-906f-4ba7-bbcb-abe01818b420,40.41,1.57
2,00059968-59b7-465f-ada9-3831759c7863,31.57,0.86
3,0005f5b5-2c8a-413f-8c38-9a54767aaf0e,131.32,1.95
4,0006ae29-03bf-4939-92b0-36294813faf5,26.18,-0.02
...,...,...,...
17846,fff9496f-29bf-4fa5-a67c-1103855c7e7d,23.64,0.50
17847,fff9d9f3-c3e1-435e-8146-85df5a6c784d,283.75,2.41
17848,fffbeb49-f2bc-46c9-94d5-9fbee3a65374,45.37,0.07
17849,fffca042-40e9-45cb-bdd2-0106ef57af6f,58.62,2.89


In [193]:
app_user_sale = pd.merge(appsflyer, average_sale_per_user, how='left', on='customer_user_id')
app_user_sale = app_user_sale.fillna(0)

In [195]:
app_user_sale.install_time = pd.to_datetime(app_user_sale.install_time)

In [197]:
app_user_sale_selected = app_user_sale[app_user_sale.install_time <= pd.Timestamp('2024-08-07')]

In [199]:
app_user_sale_selected

,customer_user_id,install_time,media_source,platform,accounting_price,order_profit
0,42627abe-9b98-408a-a146-148c6eb56529,2024-07-31 23:55:37,googleadwords_int,android,0.00,0.00
1,a833a424-5903-436f-8c4c-5984f9905a36,2024-07-30 17:53:19,googleadwords_int,android,71.23,11.83
2,96896acd-012d-41eb-ab2a-4db2c9bb4bf3,2024-07-31 23:51:28,googleadwords_int,android,0.00,0.00
3,2b4e1f45-3fa2-42d6-af72-5563a2bccd41,2024-07-31 23:48:49,Smart_Banner,android,0.00,0.00
4,af46299b-74d0-401f-98de-06580eb7e1a3,2024-07-31 23:48:49,Smart_Banner,android,0.00,0.00
...,...,...,...,...,...,...
51201,534662c8-d854-486c-a7cd-2129ab4e7c2c,2024-08-01 01:27:20,organic,ios,0.00,0.00
51202,6331aeed-a7a7-4147-9fc6-d6e0f8d61fd5,2024-08-01 01:16:21,organic,ios,0.00,0.00
51203,ad1ddc6c-68a4-47fa-9615-70ee696da947,2024-08-01 00:42:14,organic,ios,140.40,-5.22
51204,62207612-d038-425d-bee5-8ccddc952684,2024-08-01 00:57:25,organic,ios,298.71,42.99


In [203]:
app_user_sale_selected['month'] = app_user_sale_selected.loc[:, 'install_time'].dt.month

/var/folders/81/8rrb2nqs7t39v9t0vy9jm2s80000gq/T/ipykernel_69294/1682461520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_user_sale_selected['month'] = app_user_sale_selected.loc[:, 'install_time'].dt.month


In [205]:
app_user_sale_selected

,customer_user_id,install_time,media_source,platform,accounting_price,order_profit,month
0,42627abe-9b98-408a-a146-148c6eb56529,2024-07-31 23:55:37,googleadwords_int,android,0.00,0.00,7
1,a833a424-5903-436f-8c4c-5984f9905a36,2024-07-30 17:53:19,googleadwords_int,android,71.23,11.83,7
2,96896acd-012d-41eb-ab2a-4db2c9bb4bf3,2024-07-31 23:51:28,googleadwords_int,android,0.00,0.00,7
3,2b4e1f45-3fa2-42d6-af72-5563a2bccd41,2024-07-31 23:48:49,Smart_Banner,android,0.00,0.00,7
4,af46299b-74d0-401f-98de-06580eb7e1a3,2024-07-31 23:48:49,Smart_Banner,android,0.00,0.00,7
...,...,...,...,...,...,...,...
51201,534662c8-d854-486c-a7cd-2129ab4e7c2c,2024-08-01 01:27:20,organic,ios,0.00,0.00,8
51202,6331aeed-a7a7-4147-9fc6-d6e0f8d61fd5,2024-08-01 01:16:21,organic,ios,0.00,0.00,8
51203,ad1ddc6c-68a4-47fa-9615-70ee696da947,2024-08-01 00:42:14,organic,ios,140.40,-5.22,8
51204,62207612-d038-425d-bee5-8ccddc952684,2024-08-01 00:57:25,organic,ios,298.71,42.99,8


In [245]:
profit = app_user_sale_selected.groupby(['platform', 'month', 'media_source']).sum('order_profit')

In [249]:
profit

accounting_price  order_profit
platform month media_source                                  
android  7     Blog                      255.43      17.70000
               Blog_tw                   358.56      57.36960
               CRM_event                 482.07     154.01000
               Email                       0.00       0.00000
               Facebook_ads              238.21      11.13000
...                                         ...           ...
ios      8     jp_promopage                0.00       0.00000
               organic                 43365.80    3464.12839
               rd_mkt_web               5949.57     574.73000
               tw_productpage           1354.34     173.19800
               tw_promopage                0.00       0.00000

[112 rows x 2 columns]

In [237]:
counts = app_user_sale_selected[['platform', 'month', 'media_source']].groupby(
    ['platform', 'month', 'media_source']
).value_counts()

In [239]:
counts

platform  month  media_source  
android   7      Blog               15
                 Blog_tw             3
                 CRM_event          18
                 Email               1
                 Facebook_ads       23
                                  ... 
ios       8      jp_promopage        1
                 organic           885
                 rd_mkt_web         52
                 tw_productpage     16
                 tw_promopage        1
Name: count, Length: 112, dtype: int64

In [253]:
profit.to_csv('profit.csv')

In [255]:
counts.to_csv('counts.csv')